In [ ]:
!pip install opacus
!pip install datasets==2.15
!pip install peft

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import os
from opacus import PrivacyEngine
from tqdm import tqdm
from datasets import load_from_disk

In [ ]:
dataset_path = "/kaggle/input/privacy-datasets2/Privacy_datasets/mnli"

tokenized_datasets = load_from_disk(dataset_path)

In [ ]:
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=1024, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["validation_matched"], batch_size=1024)

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=4, lora_alpha=16, lora_dropout=0.1)

base_model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny",num_labels=3)

In [ ]:
model = get_peft_model(base_model, lora_config)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, eps=1e-8)

In [ ]:
model.print_trainable_parameters()

In [ ]:
model.train()
target_epsilon = 3
target_delta = 1.0 / len(tokenized_datasets["train"])
epochs = 30
max_grad_norm = 0.7

privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_epsilon=target_epsilon,
    target_delta=target_delta,
    epochs=epochs,
    max_grad_norm=max_grad_norm
)

In [ ]:
def evaluate(model):    
    model.eval()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, token_type_ids=token_type_ids)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.size(0)

    avg_loss = total_loss / len(eval_dataloader)
    accuracy = correct_predictions / total_predictions
    model.train()
    return avg_loss, accuracy
    
    

In [ ]:
from opacus.utils.batch_memory_manager import BatchMemoryManager

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(1, epochs+1):
    losses = []

    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=1024, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        for step, batch in enumerate(tqdm(memory_safe_data_loader)):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            losses.append(loss.item())

            optimizer.step()

    train_loss = np.mean(losses)
    eps = privacy_engine.get_epsilon(target_delta) if privacy_engine else None

    eval_loss, eval_accuracy = evaluate(model)

    print(
        f"Epoch: {epoch} | "
        f"Train loss: {train_loss:.3f} | "
        f"Eval loss: {eval_loss:.3f} | "
        f"Eval accuracy: {eval_accuracy:.3f} | "
        f"ɛ: {eps:.2f}" if eps is not None else ""
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for epoch in range(1, epochs+1):
    model.train()
    total_train_loss = 0
    
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, token_type_ids=token_type_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    train_loss = total_train_loss / len(train_dataloader)
    
    model.eval()
    with torch.no_grad():
        eval_loss, eval_accuracy = evaluate(model)
    
    print(
        f"Epoch: {epoch} | "
        f"Train loss: {train_loss:.3f} | "
        f"Eval loss: {eval_loss:.3f} | "
        f"Eval accuracy: {eval_accuracy:.3f} | "
    )

In [ ]:
def test(model, test_dataloader):
    model.eval()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, token_type_ids=token_type_ids)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.size(0)

    avg_loss = total_loss / len(test_dataloader)
    accuracy = correct_predictions / total_predictions

    return avg_loss, accuracy

In [ ]:
loss, accuracy = test(model, test_dataloader)

print(f"Accuracy: {test_accuracy:.3f}")